In [1]:
!uv pip install jupyterlab-vim

Using Python 3.12.9 environment at: C:\Users\RBO\repos\mbo_utilities\.venv
Audited 1 package in 93ms


In [10]:
import mbo_utilities as mbo
import numpy as np
import zarr

In [3]:
data = mbo.imread(
    r"D://demo//raw",
)
data.shape

(1574, 14, 550, 440)

In [4]:
import time

start = time.time()
mbo.imwrite(data, "./zarr", ext=".zarr", planes=[1, 2, 3], sharded=False)
end = time.time()
print(f"No sharding: {end - start}")

Saving plane01_stitched.zarr:   0%|          | 0/37 [00:00<?, ?it/s]

Saving plane02_stitched.zarr:   0%|          | 0/37 [00:00<?, ?it/s]

Saving plane03_stitched.zarr:   0%|          | 0/37 [00:00<?, ?it/s]

No sharding: 13.621733903884888


In [5]:
start = time.time()
mbo.imwrite(data, "./zarr-sharded", ext=".zarr", planes=[1, 2, 3], sharded=True)
end = time.time()
print(f"Sharding: {end - start}")

Saving plane01_stitched.zarr:   0%|          | 0/37 [00:00<?, ?it/s]

Saving plane02_stitched.zarr:   0%|          | 0/37 [00:00<?, ?it/s]

Saving plane03_stitched.zarr:   0%|          | 0/37 [00:00<?, ?it/s]

Sharding: 94.21228432655334
